In [1]:
import pandas as pd
import numpy as np
import time
import datetime

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import sklearn as sk
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
import scipy as sp
import re
import gensim
import logging
import seaborn as sns
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
%matplotlib inline
#import xgboost
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import time
from nltk.corpus import stopwords

train = pd.read_csv("data/features/features.csv",
                    header=0,delimiter=",",quotechar='"',error_bad_lines=False)

print("unfiltered data, num samples: %i with num features: %i" %(train.shape[0],train.shape[1]))

train.fillna(' ',inplace=True)
train.dropna(inplace=True)
print("filtered data with samples: %i" %(train.shape[0]))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
unfiltered data, num samples: 45885 with num features: 22
filtered data with samples: 45885


In [2]:
np.unique(train['cancer_types'])

array(["['ampullaer']", "['anus','basis']", "['anus','leber','sarkom']",
       "['anus','leukaemien-und-lymphome','leber','zervix','lunge','basis','sarkom','prostata']",
       "['anus','melanom']", "['anus','penis']", "['anus','vulva']",
       "['anus','zervix']", "['anus']", "['appendix','ovar']",
       "['appendix']", "['auge']", "['basis','hirn']",
       "['basis','hirn,gliom']", "['basis','melanom']", "['basis','net']",
       "['basis','ovar']", "['basis','pankreas']", "['basis','prostata']",
       "['basis','schilddruese']", "['basis']", "['cup']",
       "['duenndarm','appendix','net']", "['duenndarm','appendix']",
       "['duenndarm','galle','harn','leber','endometrium','pankreas']",
       "['duenndarm','galle','pankreas']",
       "['duenndarm','gist','pankreas']", "['duenndarm','gist']",
       "['duenndarm','harn','anus','kopf','hoden','ovar','sarkom','schilddruese','magen','galle','gist','oesophagus','pankreas','basaliom']",
       "['duenndarm','krk','oesophagus','

In [3]:
#y labels
useful = train["useful"]
cancer_types = train["cancer_types"]
#prepare Y
mlb = MultiLabelBinarizer()
cancer_types = mlb.fit_transform(cancer_types)

#features
abstracts = train['abstract'].values.astype(dtype=str)
title = train['fulltitle'].values.astype(dtype=str)
journal = train['subtitle'].values.astype(dtype=str)

#only rows with abstract
idxthere = np.nonzero(train['abstract'] != ' ')[0]
title = title[idxthere]
journal = journal[idxthere]
abstracts = abstracts[idxthere]
useful = useful[idxthere]

In [4]:

#preprocessing
#one hot encoded journal
le = sk.preprocessing.LabelEncoder()
enc = sk.preprocessing.OneHotEncoder()
le_journal = le.fit_transform(journal)

le_journal = le_journal.reshape(le_journal.shape[0], 1)
hot_journal = enc.fit_transform(le_journal) 
start = time.time()
#bag of words
reRemoved = [re.sub(r'[^a-z ]', '', s.lower()) for s in abstracts]
vect = CountVectorizer(max_df=0.6,min_df=10,stop_words=stopwords.words("english"))
bow_text = vect.fit_transform(reRemoved)

reRemoved = [re.sub(r'[^a-z ]', '', s.lower()) for s in title]
bow_title = vect.fit_transform(reRemoved)

In [ ]:
#only keep journals with more than 100 occurences
journal_count = np.sum(hot_journal, axis=0)
idx_imp_journals = np.nonzero(journal_count > 20)[0]
#print(idx_imp_journals.shape)
imp_journals = hot_journal[:,idx_imp_journals]
print(imp_journals)
#print(idx_imp_journals)

In [71]:
#combine features
features = sp.sparse.hstack((bow_title, bow_text, imp_journals))


In [65]:
print(features.shape)

(42868, 18749)


In [72]:
import xgboost as xgb
# read in data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, cancer_types, test_size=0.2, random_state=42)

model = xgb.XGBClassifier(max_depth=4,n_estimators=200, objective='multi:softmax')
model.fit(X_train, y_train)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=400, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [73]:
preds = model.predict_proba(X_test)[:,1]
#print(preds[:10])
print(preds.shape)
print(y_test.shape)
#np.mean(np.power(preds - y_test,2))
sk.metrics.roc_auc_score(y_test,preds)

(8574,)
(8574,)


0.81843117411110922